In [28]:
import requests
from pandas.io.json import json_normalize
from flatten_json import *
import pandas as pd
import json
import googlemaps
import time

# URL of the Google Maps API
GOOGLE_MAPS_API_URL = 'https://maps.googleapis.com/maps/api/distancematrix/json'
GOOGLE_MAPS_DIR_URL = 'https://maps.googleapis.com/maps/api/directions/json'

# Google API Key File Path; KEEP PRIVATE
api_key_file = 'Google_API_Key.txt'

# Reading the API Key
mykey = open(api_key_file,"r").readline()

# Distance Matrix API - Duration & Distance

## Required Parameters
- `origins`, can supply multiple locations separated by pipe character, in the form of address, lat/lon, or a place ID
    - `'Bobcaygeon+ON|24+Sussex+Drive+Ottawa+ON'`, `'41.43206,-81.38992|-33.86748,151.20699'`, `'47906'`
- `destinations`, same format as `origins`
- `key`, API key
***
## Optional Parameters
- `mode`, *driving*, walking, bicycling, transit
- `language`
- `region`, region code, specified as a ccTLD two-character value
- `avoid`, tolls, highways, ferries, indoor
- `units`, *metric*, imperial (only relevaqnt for the text output)
- `arrival_time` `departure_time`, desired time of arrival for transit requests, in seconds since midnight, January 1, 1970 UTC (specify either but not both, should be specified in integers)
- `traffic_model`, *best_guess*, pessimistic, optimistic
- `transit_mode`, preferred modes of transit, bus, subway, train, tram, rail
- `transit_routing_preference`, less_walking, fewer_transfers
***
## Response Elements
Each row in the response corresponds to an origin, and each element within that row corresponds to a pairing of the origin with a specific destination.
- `status`, status of the request, useful for debugging. For codes other than `OK`, look at `error_message` field for more information.
    - #### Top-level Status Codes
        - `OK`, response contains a valid `result`
        - `INVALID_REQUEST`, provided request was invalid
        - `MAX_ELEMENTS_EXCEEDED`, product of origins and destinations exceeds the per-query limit
        - `OVER_QUERY_LIMIT`, too many requests within the allowed time period
        - `REQUEST_DENIED`, service denied
        - `UNKNOWN_ERROR`, server error
    - #### Element-level Status Codes
        - `OK`, response contains a valid `result`
        - `NOT_FOUND`, origin and/or destination pairing could not be geocoded
        - `ZERO_RESULTS`, no route found
        - `MAX_ROUTE_LENGTH_EXCEEDED`, requested route is too long to process
- `origin_addresses`, array of addresses returned by the API from your original request
- `destination_addresses`, array of addresses returned by the API from your original request
- `rows`, contains an array of elements, which in turn contain a `status`, `duration`, and `distance` element
- If available, response might also contain `fare` of the route
- When `departure_time` parameter is supplied for a `driving` mode request along with valid API key, `duration_in_traffic` is also returned based upon the `traffic_model` if traffic conditions are available

In [29]:
# Sample Maps Duration and Distance using Distance Matrix API

params = {
    'units': 'imperial',
    'origins': '47906',
    'destinations': '20850',
    'key': mykey
}
try:
    req = requests.get(GOOGLE_MAPS_API_URL,params=params)
    res = req.json()
    print(json.dumps(res, sort_keys=True, indent=3))
except:
    import pdb; pdb.set_trace()
#     print(res)

# json_normalize(res)
json_normalize(flatten_json(res))

{
   "destination_addresses": [
      "Rockville, MD 20850, USA"
   ], 
   "origin_addresses": [
      "West Lafayette, IN 47906, USA"
   ], 
   "rows": [
      {
         "elements": [
            {
               "distance": {
                  "text": "641 mi", 
                  "value": 1031794
               }, 
               "duration": {
                  "text": "9 hours 40 mins", 
                  "value": 34791
               }, 
               "status": "OK"
            }
         ]
      }
   ], 
   "status": "OK"
}


,destination_addresses_0,origin_addresses_0,rows_0_elements_0_distance_text,rows_0_elements_0_distance_value,rows_0_elements_0_duration_text,rows_0_elements_0_duration_value,rows_0_elements_0_status,status
0,"Rockville, MD 20850, USA","West Lafayette, IN 47906, USA",641 mi,1031794,9 hours 40 mins,34791,OK,OK


In [30]:
params = {
    'origins': '2550+Yeager+Rd|New+York+City',
    'destinations': 'The+Bean+Chicago|NASA+Langley|Disneyworld+Florida|New+York+City',
    'mode': 'driving',
    'key': mykey
}

req1 = requests.get(GOOGLE_MAPS_API_URL, params=params)
res1 = req1.json()
res1

{u'destination_addresses': [u'201 E Randolph St, Chicago, IL 60602, USA',
  u'1 Nasa Dr, Hampton, VA 23666, USA',
  u'Walt Disney World Resort, Orlando, FL 32830, USA',
  u'New York, NY, USA'],
 u'origin_addresses': [u'2550 Yeager Rd, West Lafayette, IN 47906, USA',
  u'New York, NY, USA'],
 u'rows': [{u'elements': [{u'distance': {u'text': u'196 km', u'value': 195505},
     u'duration': {u'text': u'2 hours 6 mins', u'value': 7543},
     u'status': u'OK'},
    {u'distance': {u'text': u'1,235 km', u'value': 1234784},
     u'duration': {u'text': u'11 hours 42 mins', u'value': 42141},
     u'status': u'OK'},
    {u'distance': {u'text': u'1,676 km', u'value': 1675921},
     u'duration': {u'text': u'15 hours 1 min', u'value': 54056},
     u'status': u'OK'},
    {u'distance': {u'text': u'1,225 km', u'value': 1225219},
     u'duration': {u'text': u'11 hours 56 mins', u'value': 42975},
     u'status': u'OK'}]},
  {u'elements': [{u'distance': {u'text': u'1,273 km', u'value': 1272665},
     u'dur

In [31]:
# time.gmtime(0)
# int(time.time())

In [32]:
x = json.loads(req1.text)

for isrc, src in enumerate(x['origin_addresses']):
    for idst, dst in enumerate(x['destination_addresses']):
        row = x['rows'][isrc]
        cell = row['elements'][idst]
        if cell['status'] == 'OK':
            print('{} to {}: {}, {}.'.format(src, dst, cell['distance']['text'], cell['duration']['text']))
        else:
            print('{} to {}: status = {}'.format(src, dst, cell['status']))

2550 Yeager Rd, West Lafayette, IN 47906, USA to 201 E Randolph St, Chicago, IL 60602, USA: 196 km, 2 hours 6 mins.
2550 Yeager Rd, West Lafayette, IN 47906, USA to 1 Nasa Dr, Hampton, VA 23666, USA: 1,235 km, 11 hours 42 mins.
2550 Yeager Rd, West Lafayette, IN 47906, USA to Walt Disney World Resort, Orlando, FL 32830, USA: 1,676 km, 15 hours 1 min.
2550 Yeager Rd, West Lafayette, IN 47906, USA to New York, NY, USA: 1,225 km, 11 hours 56 mins.
New York, NY, USA to 201 E Randolph St, Chicago, IL 60602, USA: 1,273 km, 12 hours 13 mins.
New York, NY, USA to 1 Nasa Dr, Hampton, VA 23666, USA: 661 km, 6 hours 28 mins.
New York, NY, USA to Walt Disney World Resort, Orlando, FL 32830, USA: 1,765 km, 16 hours 3 mins.
New York, NY, USA to New York, NY, USA: 1 m, 1 min.


In [33]:
n_orig = params['origins'].count('|')+1
n_dest = params['destinations'].count('|')+1

# print(n_orig)

json_normalize(flatten_json(res1))

,destination_addresses_0,destination_addresses_1,destination_addresses_2,destination_addresses_3,origin_addresses_0,origin_addresses_1,rows_0_elements_0_distance_text,rows_0_elements_0_distance_value,rows_0_elements_0_duration_text,rows_0_elements_0_duration_value,...,rows_1_elements_2_distance_value,rows_1_elements_2_duration_text,rows_1_elements_2_duration_value,rows_1_elements_2_status,rows_1_elements_3_distance_text,rows_1_elements_3_distance_value,rows_1_elements_3_duration_text,rows_1_elements_3_duration_value,rows_1_elements_3_status,status
0,"201 E Randolph St, Chicago, IL 60602, USA","1 Nasa Dr, Hampton, VA 23666, USA","Walt Disney World Resort, Orlando, FL 32830, USA","New York, NY, USA","2550 Yeager Rd, West Lafayette, IN 47906, USA","New York, NY, USA",196 km,195505,2 hours 6 mins,7543,...,1765193,16 hours 3 mins,57805,OK,1 m,0,1 min,0,OK,OK


In [34]:
d = {'origin': ['47906', '2550+Yeager+Rd','NASA+Langley'],
    'destination': ['60666', '585 Purdue Mall', 'Purdue University'],
    'mode': ['driving', 'transit', 'driving']
    }
df = pd.DataFrame(data=d)
df = df[['origin','destination','mode']]
df

,origin,destination,mode
0,47906,60666,driving
1,2550+Yeager+Rd,585 Purdue Mall,transit
2,NASA+Langley,Purdue University,driving


In [35]:
# len(df)

In [36]:
gmaps = googlemaps.Client(key=mykey)
matrix = gmaps.distance_matrix(df['origin'][0], df['destination'][0], mode=df['mode'][0])
json_normalize(matrix,['rows','elements'])
# matrixdf
# matrix

,distance,duration,status
0,"{u'text': u'223 km', u'value': 222913}","{u'text': u'2 hours 12 mins', u'value': 7894}",OK


In [37]:
# for i in range(0,len(df)):
    

In [38]:
params = {
    'origins': '2550+Yeager+Rd',
    'destinations': '585 Purdue Mall, West Lafayette',
    'key': mykey,
    'mode': 'cycling',
}

req2 = requests.get(GOOGLE_MAPS_API_URL, params=params)
req2 = req2.json()
print(req2)
json_normalize(flatten_json(req2))

{u'status': u'OK', u'rows': [{u'elements': [{u'duration': {u'text': u'9 mins', u'value': 519}, u'distance': {u'text': u'3.9 km', u'value': 3855}, u'status': u'OK'}]}], u'origin_addresses': [u'2550 Yeager Rd, West Lafayette, IN 47906, USA'], u'destination_addresses': [u'585 Purdue Mall, West Lafayette, IN 47907, USA']}


,destination_addresses_0,origin_addresses_0,rows_0_elements_0_distance_text,rows_0_elements_0_distance_value,rows_0_elements_0_duration_text,rows_0_elements_0_duration_value,rows_0_elements_0_status,status
0,"585 Purdue Mall, West Lafayette, IN 47907, USA","2550 Yeager Rd, West Lafayette, IN 47906, USA",3.9 km,3855,9 mins,519,OK,OK


In [39]:
params = {
    'origins': '47906',
    'destinations': 'Wellington+Ohio',
    'key': mykey,
    'mode': 'driving',
}

req2 = requests.get(GOOGLE_MAPS_API_URL, params=params)
req2 = req2.json()
print(req2)
json_normalize(flatten_json(req2))

{u'status': u'OK', u'rows': [{u'elements': [{u'duration': {u'text': u'4 hours 53 mins', u'value': 17604}, u'distance': {u'text': u'477 km', u'value': 476982}, u'status': u'OK'}]}], u'origin_addresses': [u'West Lafayette, IN 47906, USA'], u'destination_addresses': [u'Wellington, OH 44090, USA']}


,destination_addresses_0,origin_addresses_0,rows_0_elements_0_distance_text,rows_0_elements_0_distance_value,rows_0_elements_0_duration_text,rows_0_elements_0_duration_value,rows_0_elements_0_status,status
0,"Wellington, OH 44090, USA","West Lafayette, IN 47906, USA",477 km,476982,4 hours 53 mins,17604,OK,OK


In [40]:
# Sample Maps Directions using Maps Directions API

params = {
    'units': 'imperial',
    'origin': 'Purdue+University',
    'destination': 'NASA+Langley',
    'mode': 'driving',
    'key': mykey
}

req = requests.get(GOOGLE_MAPS_DIR_URL,params=params)
res = req.json()
# print(res)
json_normalize(flatten_json(res))
# print(res)
# print(json.dumps(res, sort_keys=True, indent=3))
# import pdb; pdb.set_trace()
print(res['routes'][0]['legs'][0]['distance']['text'])
print(res['routes'][0]['legs'][0]['duration']['text'])

765 mi
11 hours 46 mins
